<a href="https://colab.research.google.com/github/elainedias16/TCC/blob/main/Word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Motivação

Word embedding é uma técnica fundamental em Processamento de Linguagem Natural (NLP) que consiste em representar palavras como vetores numéricos. Dessa forma, gerar word embeddings é essencial para criar a entrada de modelos de NLP, como tradução automática, chatbots, análise de sentimentos, sumarização, entre outros.

Anteriormente,  na tarefa de classificação, foram vistas três formas de representar palavras por meio de vetores: Vetor Binário, Frequência de Palavras e TF-IDF. A diferença entre tais métodos e os word embeddings, é que os mesmos são capazes de captar a relação entre palavras de forma mais profunda.

Dessa forma, estudar word embedding é essencial para utilizar, criar e implementar modelos de NLP na qual a base de dados textual esteja melhor representada matematicamente.

# Fundamentação teórica

# Resultados Esperados

# Referências